<a href="https://colab.research.google.com/github/YrysSuiunbaeva/Kyrgyz_MNIST/blob/main/Kyrgyz_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Распознавание кыргызских букв в рукописных текстах на кыргызском языке (Kyrgyz MNIST), с использованием нейронной сети. Этот проект был проведен The Cramer Project (thecramer.com) на [Kaggle](https://www.kaggle.com/competitions/kyrgyz-language-hand-written-letter-recognition/overview)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/train.csv')
df_train

,label,pixel_0,pixel_1,pixel_2,pixel_3,pixel_4,pixel_5,pixel_6,pixel_7,pixel_8,...,pixel_2490,pixel_2491,pixel_2492,pixel_2493,pixel_2494,pixel_2495,pixel_2496,pixel_2497,pixel_2498,pixel_2499
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80208,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
80209,20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
80210,19,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
80211,28,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
class MNISTDataset(Dataset):
    def __init__(self, features):
        self.features = features.iloc[:,1:].values.astype(np.float32)
        self.targets = torch.from_numpy(features.label.values)

    def __len__(self):
        return self.features.shape[0]

    def __getitem__(self, idx):
        features = torch.from_numpy(self.features)
        return features[idx].reshape(1, 50, 50), self.targets[idx]

In [ ]:
train_loader = DataLoader(MNISTDataset(df_train), batch_size=32, shuffle=True)

In [ ]:
class DS4_Net(nn.Module):
    def __init__(self):
        super(DS4_Net, self).__init__()

        self.feature_extractor = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1, bias = False),    # 1*224*224 -> 32*224*224   (w+1-kernel+2*padding)
            nn.GELU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.2),
            nn.Conv2d(32, 64, kernel_size=3, padding=2, groups = 32),    # 32*224*224 -> 64*226*226   (w+1-kernel+2*padding)
            nn.GELU(),
            nn.Conv2d(64, 128, kernel_size=3, padding=1, groups = 32, bias = False),    # 64*226*226 -> 32*226*226
            nn.GELU(),
            nn.BatchNorm2d(128),
            nn.Dropout(0.2),
            nn.Conv2d(128, 128, kernel_size=5, stride=2),    # 32*226*226 -> 64*111*111
            nn.GELU(),
            nn.Conv2d(128, 256, kernel_size=3, padding=1, groups = 64, bias = False),    # 64*111*111 -> 64*111*111
            nn.GELU(),
            nn.BatchNorm2d(256),
            nn.Dropout(0.2),
            nn.Conv2d(256, 512, kernel_size=5, padding=1, groups = 32),    # 64*111*111 -> 32*109*109
            nn.GELU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=2, groups = 32),    #32*109*109 -> 64*111*11
            nn.GELU(),
            nn.BatchNorm2d(512),
            nn.Dropout(0.2),
            nn.Conv2d(512, 1024, kernel_size=2, stride=2, bias = False),    #64*111*111 -> 128*55*55
            nn.GELU(),
            nn.BatchNorm2d(1024),
            nn.Conv2d(1024, 2048, kernel_size=2, padding=1, groups = 128),   #128*55*55 -> 32*56*56
            nn.GELU(),
            nn.Conv2d(2048, 2048, kernel_size=3, padding=1, groups = 256),    #32*56*56 -> 32*56*56
            nn.GELU()
        )

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.3),
            nn.Linear(2048, 37)
        )

    def forward(self, x):
        x = self.feature_extractor(x)
        x = self.avgpool(x)
        x = self.classifier(x)
        return x

In [ ]:
x, y = next(iter(train_loader))

In [ ]:
model = DS4_Net()
model.to(device)

DS4_Net(
  (feature_extractor): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): GELU(approximate='none')
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.2, inplace=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2), groups=32)
    (5): GELU(approximate='none')
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
    (7): GELU(approximate='none')
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Dropout(p=0.2, inplace=False)
    (10): Conv2d(128, 128, kernel_size=(5, 5), stride=(2, 2))
    (11): GELU(approximate='none')
    (12): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
    (13): GELU(approximate='none')
    (14): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  

In [ ]:
!pip install torchsummary

In [ ]:
from torchsummary import summary
summary(model, (1, 50, 50))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 50, 50]             288
              GELU-2           [-1, 32, 50, 50]               0
       BatchNorm2d-3           [-1, 32, 50, 50]              64
           Dropout-4           [-1, 32, 50, 50]               0
            Conv2d-5           [-1, 64, 52, 52]             640
              GELU-6           [-1, 64, 52, 52]               0
            Conv2d-7          [-1, 128, 52, 52]           2,304
              GELU-8          [-1, 128, 52, 52]               0
       BatchNorm2d-9          [-1, 128, 52, 52]             256
          Dropout-10          [-1, 128, 52, 52]               0
           Conv2d-11          [-1, 128, 24, 24]         409,728
             GELU-12          [-1, 128, 24, 24]               0
           Conv2d-13          [-1, 256, 24, 24]           4,608
             GELU-14          [-1, 256,

In [ ]:
from tqdm import tqdm

def train_fn(model, optimizer, loss_fn, dataloader, device, epoch):
    model.train()
    final_acc = 0
    final_loss = 0
    total = 0
    loop = tqdm(dataloader)
    for x, y in loop:
        x, y = x.to(device).float(), y.to(device)

        outputs = model(x)
        loss = loss_fn(outputs, y)
        final_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs, 1)
        final_acc += ((predicted==y).sum().item())
        total += len(y)

        loop.set_description(f'Epoch: [{epoch + 1}/{10}]')
        loop.set_postfix(loss =(final_loss/len(dataloader)), acc = (final_acc/total))

In [ ]:
def run_training():
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.CrossEntropyLoss()

    for epoch in range(50):
        train_fn(model, optimizer, loss_fn, train_loader, device, epoch)

In [ ]:
run_training()

Epoch: [50/10]: 100%|██████████| 2507/2507 [04:38<00:00,  9.01it/s, acc=0.997, loss=0.0106]


In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/test.csv')
df_test

,id,pixel_0,pixel_1,pixel_2,pixel_3,pixel_4,pixel_5,pixel_6,pixel_7,pixel_8,...,pixel_2490,pixel_2491,pixel_2492,pixel_2493,pixel_2494,pixel_2495,pixel_2496,pixel_2497,pixel_2498,pixel_2499
0,276f619c,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8ced6677,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,af549c74,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,59170e1b,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ac25b7f0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8899,7ee547cf,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8900,dfc2f44d,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8901,543715a7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8902,47d51e67,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
x = df_test.iloc[:,1:]
x.head()

,pixel_0,pixel_1,pixel_2,pixel_3,pixel_4,pixel_5,pixel_6,pixel_7,pixel_8,pixel_9,...,pixel_2490,pixel_2491,pixel_2492,pixel_2493,pixel_2494,pixel_2495,pixel_2496,pixel_2497,pixel_2498,pixel_2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
class MNISTTestDataset(Dataset):
    def __init__(self, features):
        self.features = features.values.astype(np.float32)

    def __len__(self):
        return self.features.shape[0]

    def __getitem__(self, idx):
        features = torch.from_numpy(self.features)
        return features[idx].reshape(1, 50, 50)

In [ ]:
test_loader = DataLoader(MNISTTestDataset(x), batch_size=32, shuffle=False)

In [ ]:
def test_fn(model, dataloader, device):
    model.eval()
    predictions = []
    with torch.no_grad():
        for i in tqdm(dataloader):
            i = i.to(device).float()
            outputs = model(i)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
    return predictions


In [ ]:
predictions = test_fn(model, test_loader, device)

100%|██████████| 279/279 [00:11<00:00, 24.41it/s]


In [ ]:
df_test['label'] = predictions
df_test[['id','label']].to_csv('letters_epoch_50_2048.csv', index=False)

In [ ]:
df_test[['id','label']]

,id,label
0,276f619c,21
1,8ced6677,11
2,af549c74,21
3,59170e1b,31
4,ac25b7f0,19
...,...,...
8899,7ee547cf,2
8900,dfc2f44d,29
8901,543715a7,34
8902,47d51e67,7
